In [1]:
from xmitgcm import llcreader
model = llcreader.ECCOPortalLLC4320Model()
import numpy as np
import pandas as pd
import xarray as xr
import os
from datetime import datetime

CASE_NAME = 'california/'
BASE = '/scratch/ab10313/submeso_ML_data/'
PATH = BASE+CASE_NAME


# select subdomain area and downsample every 24 hours to reduce auto-correlation between samples (Uchida et. al. 2022)
# depth above -700m (as we will average over the mixed layer below)
# lat/lon domain to be larger than 12 degrees

lat_min = 30
lat_max = 45
lon_min = -140
lon_max = -125
depth_lim = -700


In [5]:
sel_area_T_surf = np.logical_and(np.logical_and(ds_HBL_full.XC>lon_min, ds_HBL_full.XC<lon_max ),np.logical_and(ds_HBL_full.YC>lat_min, ds_HBL_full.YC<lat_max))

In [2]:
ds_HBL_full = model.get_dataset(varnames=['KPPhbl'], type='latlon')

In [7]:
ds_HBL = ds_HBL_full.KPPhbl.where(sel_area_T_surf, drop=True)

FileNotFoundError: https://data.nas.nasa.gov/ecco/download_data.php?file=/eccodata/llc_4320/grid/XC.data

In [4]:
ds_HBL_full.KPPhbl

<xarray.DataArray 'KPPhbl' (time: 10312, j: 12960, i: 17280)>
dask.array<concatenate, shape=(10312, 12960, 17280), dtype=float32, chunksize=(1, 12960, 4320), chunktype=numpy.ndarray>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 ... 17274 17275 17276 17277 17278 17279
  * j        (j) int64 0 1 2 3 4 5 6 ... 12954 12955 12956 12957 12958 12959
  * time     (time) datetime64[ns] 2011-09-13 ... 2012-11-15T15:00:00
    CS       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    SN       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    Depth    (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    dxF      (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    dyF      (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    rA       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    XC       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    YC       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
Attributes:
    standard_name:  KPP_boundary_layer
    long_name:      KPP boundary layer depth, bulk Ri criterion
    units:          m

In [10]:
ds_Q_full = model.get_dataset(varnames=['oceQnet'], type='latlon')
ds_Q_full.oceQnet

<xarray.DataArray 'oceQnet' (time: 10312, j: 12960, i: 17280)>
dask.array<concatenate, shape=(10312, 12960, 17280), dtype=float32, chunksize=(1, 12960, 4320), chunktype=numpy.ndarray>
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 ... 17274 17275 17276 17277 17278 17279
  * j        (j) int64 0 1 2 3 4 5 6 ... 12954 12955 12956 12957 12958 12959
  * time     (time) datetime64[ns] 2011-09-13 ... 2012-11-15T15:00:00
    CS       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    SN       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    Depth    (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    dxF      (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    dyF      (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    rA       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    XC       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    YC       (j, i) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
Attributes:
    standard_name:  oceQnet
    long_name:      net surface heat flux into the ocean (+=down), >0 increas...
    units:          W/m^2

In [8]:
ds_UV_full = model.get_dataset(varnames=['U','V'], type='latlon')

In [9]:
ds_UV_full.U

<xarray.DataArray 'U' (time: 10312, k: 90, j: 12960, i_g: 17280)>
dask.array<concatenate, shape=(10312, 90, 12960, 17280), dtype=float32, chunksize=(1, 1, 12960, 4320), chunktype=numpy.ndarray>
Coordinates:
  * i_g      (i_g) int64 0 1 2 3 4 5 6 ... 17274 17275 17276 17277 17278 17279
  * j        (j) int64 0 1 2 3 4 5 6 ... 12954 12955 12956 12957 12958 12959
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * time     (time) datetime64[ns] 2011-09-13 ... 2012-11-15T15:00:00
    drF      (k) >f4 dask.array<chunksize=(90,), meta=np.ndarray>
    dxC      (j, i_g) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    dyG      (j, i_g) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    hFacW    (k, j, i_g) float32 dask.array<chunksize=(1, 12960, 4320), meta=np.ndarray>
    PHrefC   (k) >f4 dask.array<chunksize=(90,), meta=np.ndarray>
    rAw      (j, i_g) float32 dask.array<chunksize=(12960, 4320), meta=np.ndarray>
    Z        (k) >f4 dask.array<chunksize=(90,), meta=np.ndarray>
    rhoRef   (k) >f4 dask.array<chunksize=(90,), meta=np.ndarray>
Attributes:
    standard_name:  sea_water_x_velocity
    mate:           V
    long_name:      Zonal Component of Velocity
    units:          m s-1

In [56]:
print('CALCULATE FCOR')

omega = 7.2921e-5

lat = np.arange(lat_min,lat_max,0.5)
lon = np.arange(lon_min,lon_max,1)

FCOR = xr.DataArray(np.zeros((len(lon),len(lat))))


for ii in range(len(lat)):
        FCOR[:,ii] = 2*omega*np.sin(lat[ii]* np.pi / 180.)


print('COMPLETE PREPROCESS')
print(datetime.now())

CALCULATE FCOR
COMPLETE PREPROCESS
2023-01-11 12:33:14.343690


In [59]:
FCOR

<xarray.DataArray (dim_0: 15, dim_1: 30)>
array([[7.29210000e-05, 7.40204099e-05, 7.51141829e-05, 7.62022357e-05,
        7.72844853e-05, 7.83608495e-05, 7.94312461e-05, 8.04955938e-05,
        8.15538114e-05, 8.26058184e-05, 8.36515346e-05, 8.46908805e-05,
        8.57237768e-05, 8.67501449e-05, 8.77699066e-05, 8.87829843e-05,
        8.97893009e-05, 9.07887796e-05, 9.17813445e-05, 9.27669198e-05,
        9.37454306e-05, 9.47168023e-05, 9.56809609e-05, 9.66378331e-05,
        9.75873459e-05, 9.85294271e-05, 9.94640048e-05, 1.00391008e-04,
        1.01310366e-04, 1.02222009e-04],
       [7.29210000e-05, 7.40204099e-05, 7.51141829e-05, 7.62022357e-05,
        7.72844853e-05, 7.83608495e-05, 7.94312461e-05, 8.04955938e-05,
        8.15538114e-05, 8.26058184e-05, 8.36515346e-05, 8.46908805e-05,
        8.57237768e-05, 8.67501449e-05, 8.77699066e-05, 8.87829843e-05,
        8.97893009e-05, 9.07887796e-05, 9.17813445e-05, 9.27669198e-05,
        9.37454306e-05, 9.47168023e-05, 9.56809609e-05, 9.66378331e-05,
        9.75873459e-05, 9.85294271e-05, 9.94640048e-05, 1.00391008e-04,
        1.01310366e-04, 1.02222009e-04],
       [7.29210000e-05, 7.40204099e-05, 7.51141829e-05, 7.62022357e-05,
        7.72844853e-05, 7.83608495e-05, 7.94312461e-05, 8.04955938e-05,
        8.15538114e-05, 8.26058184e-05, 8.36515346e-05, 8.46908805e-05,
        8.57237768e-05, 8.67501449e-05, 8.77699066e-05, 8.87829843e-05,
...
        8.97893009e-05, 9.07887796e-05, 9.17813445e-05, 9.27669198e-05,
        9.37454306e-05, 9.47168023e-05, 9.56809609e-05, 9.66378331e-05,
        9.75873459e-05, 9.85294271e-05, 9.94640048e-05, 1.00391008e-04,
        1.01310366e-04, 1.02222009e-04],
       [7.29210000e-05, 7.40204099e-05, 7.51141829e-05, 7.62022357e-05,
        7.72844853e-05, 7.83608495e-05, 7.94312461e-05, 8.04955938e-05,
        8.15538114e-05, 8.26058184e-05, 8.36515346e-05, 8.46908805e-05,
        8.57237768e-05, 8.67501449e-05, 8.77699066e-05, 8.87829843e-05,
        8.97893009e-05, 9.07887796e-05, 9.17813445e-05, 9.27669198e-05,
        9.37454306e-05, 9.47168023e-05, 9.56809609e-05, 9.66378331e-05,
        9.75873459e-05, 9.85294271e-05, 9.94640048e-05, 1.00391008e-04,
        1.01310366e-04, 1.02222009e-04],
       [7.29210000e-05, 7.40204099e-05, 7.51141829e-05, 7.62022357e-05,
        7.72844853e-05, 7.83608495e-05, 7.94312461e-05, 8.04955938e-05,
        8.15538114e-05, 8.26058184e-05, 8.36515346e-05, 8.46908805e-05,
        8.57237768e-05, 8.67501449e-05, 8.77699066e-05, 8.87829843e-05,
        8.97893009e-05, 9.07887796e-05, 9.17813445e-05, 9.27669198e-05,
        9.37454306e-05, 9.47168023e-05, 9.56809609e-05, 9.66378331e-05,
        9.75873459e-05, 9.85294271e-05, 9.94640048e-05, 1.00391008e-04,
        1.01310366e-04, 1.02222009e-04]])
Dimensions without coordinates: dim_0, dim_1

In [58]:
FCOR*FCOR

<xarray.DataArray (dim_0: 15, dim_1: 30)>
array([[5.31747224e-09, 5.47902109e-09, 5.64214048e-09, 5.80678072e-09,
        5.97289167e-09, 6.14042273e-09, 6.30932286e-09, 6.47954062e-09,
        6.65102416e-09, 6.82372123e-09, 6.99757925e-09, 7.17254523e-09,
        7.34856590e-09, 7.52558763e-09, 7.70355651e-09, 7.88241831e-09,
        8.06211855e-09, 8.24260251e-09, 8.42381519e-09, 8.60570141e-09,
        8.78820575e-09, 8.97127263e-09, 9.15484628e-09, 9.33887078e-09,
        9.52329008e-09, 9.70804800e-09, 9.89308826e-09, 1.00783545e-08,
        1.02637903e-08, 1.04493391e-08],
       [5.31747224e-09, 5.47902109e-09, 5.64214048e-09, 5.80678072e-09,
        5.97289167e-09, 6.14042273e-09, 6.30932286e-09, 6.47954062e-09,
        6.65102416e-09, 6.82372123e-09, 6.99757925e-09, 7.17254523e-09,
        7.34856590e-09, 7.52558763e-09, 7.70355651e-09, 7.88241831e-09,
        8.06211855e-09, 8.24260251e-09, 8.42381519e-09, 8.60570141e-09,
        8.78820575e-09, 8.97127263e-09, 9.15484628e-09, 9.33887078e-09,
        9.52329008e-09, 9.70804800e-09, 9.89308826e-09, 1.00783545e-08,
        1.02637903e-08, 1.04493391e-08],
       [5.31747224e-09, 5.47902109e-09, 5.64214048e-09, 5.80678072e-09,
        5.97289167e-09, 6.14042273e-09, 6.30932286e-09, 6.47954062e-09,
        6.65102416e-09, 6.82372123e-09, 6.99757925e-09, 7.17254523e-09,
        7.34856590e-09, 7.52558763e-09, 7.70355651e-09, 7.88241831e-09,
...
        8.06211855e-09, 8.24260251e-09, 8.42381519e-09, 8.60570141e-09,
        8.78820575e-09, 8.97127263e-09, 9.15484628e-09, 9.33887078e-09,
        9.52329008e-09, 9.70804800e-09, 9.89308826e-09, 1.00783545e-08,
        1.02637903e-08, 1.04493391e-08],
       [5.31747224e-09, 5.47902109e-09, 5.64214048e-09, 5.80678072e-09,
        5.97289167e-09, 6.14042273e-09, 6.30932286e-09, 6.47954062e-09,
        6.65102416e-09, 6.82372123e-09, 6.99757925e-09, 7.17254523e-09,
        7.34856590e-09, 7.52558763e-09, 7.70355651e-09, 7.88241831e-09,
        8.06211855e-09, 8.24260251e-09, 8.42381519e-09, 8.60570141e-09,
        8.78820575e-09, 8.97127263e-09, 9.15484628e-09, 9.33887078e-09,
        9.52329008e-09, 9.70804800e-09, 9.89308826e-09, 1.00783545e-08,
        1.02637903e-08, 1.04493391e-08],
       [5.31747224e-09, 5.47902109e-09, 5.64214048e-09, 5.80678072e-09,
        5.97289167e-09, 6.14042273e-09, 6.30932286e-09, 6.47954062e-09,
        6.65102416e-09, 6.82372123e-09, 6.99757925e-09, 7.17254523e-09,
        7.34856590e-09, 7.52558763e-09, 7.70355651e-09, 7.88241831e-09,
        8.06211855e-09, 8.24260251e-09, 8.42381519e-09, 8.60570141e-09,
        8.78820575e-09, 8.97127263e-09, 9.15484628e-09, 9.33887078e-09,
        9.52329008e-09, 9.70804800e-09, 9.89308826e-09, 1.00783545e-08,
        1.02637903e-08, 1.04493391e-08]])
Dimensions without coordinates: dim_0, dim_1

In [30]:
lat_lon

TypeError: unsupported format string passed to numpy.ndarray.__format__

<xarray.DataArray (dim_0: 2)>
array([array([30. , 30.5, 31. , 31.5, 32. , 32.5, 33. , 33.5, 34. , 34.5, 35. ,
              35.5, 36. , 36.5, 37. , 37.5, 38. , 38.5, 39. , 39.5, 40. , 40.5,
              41. , 41.5, 42. , 42.5, 43. , 43.5, 44. , 44.5])                 ,
       array([-140, -139, -138, -137, -136, -135, -134, -133, -132, -131, -130,
              -129, -128, -127, -126])                                         ],
      dtype=object)
Dimensions without coordinates: dim_0

In [24]:
lat_lon

<xarray.DataArray (dim_0: 2, dim_1: 15)>
array([[  30,   31,   32,   33,   34,   35,   36,   37,   38,   39,   40,
          41,   42,   43,   44],
       [-140, -139, -138, -137, -136, -135, -134, -133, -132, -131, -130,
        -129, -128, -127, -126]])
Dimensions without coordinates: dim_0, dim_1